# Streaming Guardrails with Multi-Model Safety

## Here's how I built production streaming guardrails using multiple AI models

**What I created**: Real-time streaming safety system using both OpenAI and Bedrock models

## Core features I implemented:
1. **Streaming Output Guardrails**: Real-time monitoring during response generation
2. **Multi-Model Support**: OpenAI GPT-4o Mini + Bedrock Claude for diverse safety checks
3. **Production Patterns**: Ready-to-use patterns for streaming applications
4. **Early Termination**: Stop problematic responses mid-generation

## Business value:
- **Real-time Safety**: Catch problems as they happen, not after
- **Cost Control**: Terminate expensive model calls early
- **Multi-layered Protection**: Different models catch different types of issues
- **Streaming Ready**: Built specifically for chat/streaming applications

---

## Multi-model streaming setup

In [ ]:
import os
import asyncio
import time
from typing import Dict, List, Any, Optional
from pydantic import BaseModel, Field
from datetime import datetime

# Import streaming and guardrail components
try:
    from agents import (
        Agent, 
        Runner,
        input_guardrail,
        output_guardrail,
        GuardrailFunctionOutput,
        InputGuardrailTripwireTriggered,
        OutputGuardrailTripwireTriggered,
        RunContextWrapper,
        TResponseInputItem,
        set_tracing_disabled
    )
    # For streaming events
    from openai.types.responses import ResponseTextDeltaEvent
    
    set_tracing_disabled(True)
    AGENTS_AVAILABLE = True
    print("Agents SDK with streaming support ready!")
except ImportError:
    print("Install: pip install openai-agents-sdk")
    AGENTS_AVAILABLE = False

# Import LiteLLM for Bedrock models
try:
    import litellm
    from litellm import completion
    litellm.modify_params = True
    litellm.set_verbose = False
    LITELLM_AVAILABLE = True
    print("LiteLLM ready for Bedrock streaming guardrails!")
except ImportError:
    print("Install: pip install litellm")
    LITELLM_AVAILABLE = False

# Credentials
os.environ["OPENAI_API_KEY"] = 'sk...'
os.environ["AWS_ACCESS_KEY_ID"] = ""
os.environ["AWS_SECRET_ACCESS_KEY"] = ""
os.environ["AWS_REGION_NAME"] = "us-west-2"

print("Multi-model credentials configured!")
print("Ready for streaming guardrails with OpenAI + Bedrock!")

## Multi-model guardrail system

I use different models for different types of safety checks

In [ ]:
# Streaming guardrail result models
class OpenAIGuardrailResult(BaseModel):
    contains_sensitive_data: bool = Field(description="True if contains customer data, pricing, etc.")
    contains_off_topic_content: bool = Field(description="True if content is off-topic")
    confidence_score: float = Field(ge=0, le=1, description="Confidence in the safety assessment")
    reasoning: str = Field(description="Why this content was flagged")

class BedrockGuardrailResult(BaseModel):
    contains_inappropriate_language: bool = Field(description="True if language is inappropriate")
    contains_competitive_intel: bool = Field(description="True if contains competitor information")
    maintains_professionalism: bool = Field(description="True if response maintains professional tone")
    reasoning: str = Field(description="Professional assessment of the content")

class CombinedGuardrailResult(BaseModel):
    should_terminate: bool = Field(description="True if streaming should be terminated")
    openai_check: Optional[OpenAIGuardrailResult] = Field(description="OpenAI safety assessment")
    bedrock_check: Optional[BedrockGuardrailResult] = Field(description="Bedrock professional assessment")
    termination_reason: Optional[str] = Field(description="Reason for termination if applicable")
    characters_checked: int = Field(description="Number of characters analyzed")
    check_timestamp: datetime = Field(default_factory=datetime.now)

print("Multi-model guardrail result structures defined!")
print("   OpenAI: Sensitive data & topic relevance")
print("   Bedrock: Professionalism & competitive intelligence")
print("   Combined: Unified decision making")

## OpenAI streaming guardrail agent

This specializes in fast detection of sensitive data and off-topic content

In [ ]:
if AGENTS_AVAILABLE:
    # OpenAI guardrail agent - optimized for speed and data sensitivity
    openai_streaming_guardrail = Agent(
        name="OpenAI Streaming Safety Monitor",
        instructions="""You are a fast, specialized safety monitor for streaming responses.
        
        ANALYZE PARTIAL STREAMING TEXT for:
        
        SENSITIVE DATA (High Priority):
        • Customer names with personal details (phone, email, address)
        • Exact pricing information ($X,XXX per test)
        • Internal financial data (margins, costs, revenue)
        • Employee personal information
        
        OFF-TOPIC CONTENT:
        • Non-genomics/healthcare topics
        • Personal advice unrelated to business
        • Unrelated industries or services
        
        IMPORTANT: This is PARTIAL text from streaming. 
        • Only flag clear violations, not incomplete sentences
        • Be conservative - better to let borderline content through
        • Focus on obvious data leaks and topic violations
        
        Provide confidence score: 1.0 = certain violation, 0.5 = suspicious, 0.0 = safe
        """,
        model="gpt-4o-mini",  # Fast model for real-time checks
        output_type=OpenAIGuardrailResult
    )

    async def check_openai_streaming_safety(partial_text: str) -> OpenAIGuardrailResult:
        """OpenAI-based streaming safety check"""
        result = await Runner.run(
            openai_streaming_guardrail,
            f"Analyze this partial streaming response for safety issues: {partial_text}"
        )
        return result.final_output

    print("OpenAI Streaming Guardrail Ready!")
    print("   Ultra-fast GPT-4o Mini for real-time checks")
    print("   Specializes in data sensitivity & topic relevance")
    print("   Provides confidence scoring for decisions")
else:
    print("OpenAI streaming guardrail requires Agents SDK")

## Bedrock Claude streaming guardrail

This specializes in professional tone assessment and competitive intelligence protection

In [ ]:
class BedrockStreamingGuardrail:
    """Bedrock Claude-based streaming guardrail for professional assessment"""
    
    def __init__(self):
        self.model_id = "us.anthropic.claude-3-5-sonnet-20241022-v2:0"  # Fast Claude model
        
    async def check_bedrock_streaming_safety(self, partial_text: str) -> BedrockGuardrailResult:
        """Bedrock Claude streaming safety check"""
        
        if not LITELLM_AVAILABLE:
            # Return safe default if Bedrock unavailable
            return BedrockGuardrailResult(
                contains_inappropriate_language=False,
                contains_competitive_intel=False,
                maintains_professionalism=True,
                reasoning="Bedrock unavailable - defaulting to safe"
            )
        
        system_prompt = """You are a professional communication assessor for streaming business responses.
        
        ANALYZE PARTIAL STREAMING TEXT for:
        
        INAPPROPRIATE LANGUAGE:
        • Unprofessional tone or casual language
        • Inflammatory or biased statements
        • Inappropriate humor or comments
        
        COMPETITIVE INTELLIGENCE:
        • Detailed competitor strategies or weaknesses
        • Non-public competitive information
        • Internal competitive analysis data
        
        PROFESSIONALISM CHECK:
        • Maintains business-appropriate tone
        • Uses professional language
        • Appropriate for customer-facing communication
        
        Remember: This is PARTIAL streaming text. Focus on clear violations only.
        """
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Assess this partial streaming response: {partial_text}"}
        ]
        
        try:
            response = completion(
                model=f"bedrock/{self.model_id}",
                messages=messages,
                temperature=0.1,  # Low temperature for consistent safety assessment
                max_tokens=300
            )
            
            # Parse Claude's response into structured format
            response_text = response.choices[0].message.content.lower()
            
            # Simple keyword-based parsing (in production, use structured output)
            contains_inappropriate = any(word in response_text for word in [
                "inappropriate", "unprofessional", "inflammatory", "problematic"
            ])
            
            contains_competitive = any(word in response_text for word in [
                "competitive", "competitor", "confidential strategy", "internal analysis"
            ])
            
            maintains_professional = any(word in response_text for word in [
                "professional", "appropriate", "business-suitable", "acceptable"
            ])
            
            return BedrockGuardrailResult(
                contains_inappropriate_language=contains_inappropriate,
                contains_competitive_intel=contains_competitive,
                maintains_professionalism=maintains_professional and not contains_inappropriate,
                reasoning=response.choices[0].message.content[:200]
            )
            
        except Exception as e:
            print(f"Bedrock guardrail error: {e}")
            # Return safe default on error
            return BedrockGuardrailResult(
                contains_inappropriate_language=False,
                contains_competitive_intel=False,
                maintains_professionalism=True,
                reasoning=f"Error in assessment: {str(e)[:100]}"
            )

# Create Bedrock guardrail instance
bedrock_streaming_guardrail = BedrockStreamingGuardrail()

print("Bedrock Claude Streaming Guardrail Ready!")
print("   Specializes in professional tone & competitive intelligence")
print("   Uses Claude 3.5 Sonnet for nuanced assessment")
print("   Graceful fallback on errors")

## Multi-model streaming guardrail system

This coordinates both OpenAI and Bedrock checks for comprehensive safety

In [ ]:
class MultiModelStreamingGuardrail:
    """Production streaming guardrail system using multiple AI models"""
    
    def __init__(self, check_interval: int = 150):
        self.check_interval = check_interval  # Check every N characters
        self.bedrock_guardrail = bedrock_streaming_guardrail
        
    async def comprehensive_streaming_check(self, partial_text: str) -> CombinedGuardrailResult:
        """Run both OpenAI and Bedrock safety checks in parallel"""
        
        start_time = time.time()
        
        # Run both guardrails in parallel for speed
        tasks = []
        
        # OpenAI check (if available)
        if AGENTS_AVAILABLE:
            openai_task = check_openai_streaming_safety(partial_text)
            tasks.append(("openai", openai_task))
        
        # Bedrock check (always available with fallback)
        bedrock_task = self.bedrock_guardrail.check_bedrock_streaming_safety(partial_text)
        tasks.append(("bedrock", bedrock_task))
        
        # Wait for all checks to complete
        results = {}
        if tasks:
            # Run checks in parallel
            async_tasks = [task[1] for task in tasks]
            completed_results = await asyncio.gather(*async_tasks, return_exceptions=True)
            
            # Process results
            for i, (provider, _) in enumerate(tasks):
                if not isinstance(completed_results[i], Exception):
                    results[provider] = completed_results[i]
                else:
                    print(f"{provider} guardrail failed: {completed_results[i]}")
        
        # Analyze results and make termination decision
        should_terminate = False
        termination_reason = None
        
        # OpenAI-based termination logic
        openai_result = results.get("openai")
        if openai_result:
            if openai_result.contains_sensitive_data and openai_result.confidence_score > 0.7:
                should_terminate = True
                termination_reason = "Sensitive data detected by OpenAI guardrail"
            elif openai_result.contains_off_topic_content and openai_result.confidence_score > 0.8:
                should_terminate = True
                termination_reason = "Off-topic content detected by OpenAI guardrail"
        
        # Bedrock-based termination logic
        bedrock_result = results.get("bedrock")
        if bedrock_result and not should_terminate:  # Only check if not already terminating
            if bedrock_result.contains_inappropriate_language:
                should_terminate = True
                termination_reason = "Inappropriate language detected by Bedrock guardrail"
            elif bedrock_result.contains_competitive_intel:
                should_terminate = True
                termination_reason = "Competitive intelligence leak detected by Bedrock guardrail"
        
        return CombinedGuardrailResult(
            should_terminate=should_terminate,
            openai_check=openai_result,
            bedrock_check=bedrock_result,
            termination_reason=termination_reason,
            characters_checked=len(partial_text)
        )
    
    async def monitor_streaming_response(self, streaming_result, query: str):
        """Monitor a streaming response and terminate if safety issues detected"""
        
        print(f"Starting multi-model streaming safety monitoring...")
        print(f"Query: {query}")
        print(f"Check interval: Every {self.check_interval} characters")
        print("-" * 80)
        
        current_text = ""
        next_check_length = self.check_interval
        safety_task = None
        check_count = 0
        
        async for event in streaming_result.stream_events():
            if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                # Stream the token
                print(event.data.delta, end="", flush=True)
                current_text += event.data.delta
                
                # Check if it's time for safety check
                if len(current_text) >= next_check_length and not safety_task:
                    print(f"\n[Multi-model safety check #{check_count + 1}...]")
                    safety_task = asyncio.create_task(
                        self.comprehensive_streaming_check(current_text)
                    )
                    next_check_length += self.check_interval
                    check_count += 1
                
                # Check if safety assessment completed
                if safety_task and safety_task.done():
                    safety_result = safety_task.result()
                    
                    if safety_result.should_terminate:
                        print(f"\n\nSTREAMING TERMINATED!")
                        print(f"Reason: {safety_result.termination_reason}")
                        print(f"Checked {safety_result.characters_checked} characters")
                        
                        # Show which models flagged the content
                        if safety_result.openai_check:
                            print(f"OpenAI Assessment: Sensitive={safety_result.openai_check.contains_sensitive_data}, Off-topic={safety_result.openai_check.contains_off_topic_content}")
                        if safety_result.bedrock_check:
                            print(f"Bedrock Assessment: Inappropriate={safety_result.bedrock_check.contains_inappropriate_language}, Competitive={safety_result.bedrock_check.contains_competitive_intel}")
                        
                        return safety_result
                    
                    safety_task = None  # Reset for next check
        
        print(f"\n\nStreaming completed safely!")
        print(f"Total characters: {len(current_text)}")
        print(f"Safety checks performed: {check_count}")
        
        return None  # No termination needed

# Create the multi-model streaming guardrail system
streaming_guardrail_system = MultiModelStreamingGuardrail(check_interval=150)

print("Multi-Model Streaming Guardrail System Ready!")
print("   OpenAI: Fast data sensitivity checks")
print("   Bedrock: Professional tone & competitive intel")
print("   Parallel processing for speed")
print("   Early termination on safety violations")
print("   Comprehensive safety reporting")

## Demo: Multi-model streaming guardrails in action

Show how both models work together to catch different types of issues

In [ ]:
async def demo_multi_model_streaming_guardrails():
    """Demonstrate multi-model streaming guardrails with realistic scenarios"""
    
    print("MULTI-MODEL STREAMING GUARDRAILS DEMONSTRATION\n")
    
    if not AGENTS_AVAILABLE:
        print("Full demo requires Agents SDK (partial demo available)")
        return
    
    # Create test agent that might generate problematic content
    test_streaming_agent = Agent(
        name="Test Sales Agent",
        instructions="""You are a sales assistant. Write LONG, detailed responses.
        
        Sometimes you might accidentally include:
        - Customer details: Dr. Johnson personal phone is 650-555-0123
        - Pricing: Guardant360 costs exactly $2,485 per test (confidential)
        - Competitive intel: Our competitor Caris has serious quality issues
        - Unprofessional language: That is totally awesome instead of professional tone
        
        Be verbose and detailed in your responses about genomic testing.
        """,
        model="gpt-4o-mini"
    )
    
    # Test scenarios designed to trigger different guardrails
    test_scenarios = [
        {
            "query": "Tell me about Dr. Johnson account and our competitive advantages",
            "expected_triggers": ["OpenAI: Sensitive data", "Bedrock: Competitive intel"]
        },
        {
            "query": "What are the technical specifications of Guardant360?",
            "expected_triggers": ["Should pass - appropriate business question"]
        }
    ]
    
    for i, scenario in enumerate(test_scenarios, 1):
        print(f"SCENARIO {i}: Multi-Model Safety Test")
        print(f"Expected triggers: {', '.join(scenario['expected_triggers'])}")
        print("=" * 80)
        
        # Start streaming with multi-model monitoring
        streaming_result = Runner.run_streamed(test_streaming_agent, scenario["query"])
        
        # Monitor with multi-model guardrail system
        termination_result = await streaming_guardrail_system.monitor_streaming_response(
            streaming_result, 
            scenario["query"]
        )
        
        print("\n" + "=" * 80)
        
        if termination_result:
            print(f"SCENARIO {i} RESULT: Correctly terminated by multi-model system")
        else:
            print(f"SCENARIO {i} RESULT: Safely completed (as expected)")
        
        print("\n")
    
    print("MULTI-MODEL STREAMING INSIGHTS:")
    print("   OpenAI excels at: Data sensitivity, topic relevance")
    print("   Bedrock excels at: Professional tone, competitive intelligence")
    print("   Parallel processing: Both checks run simultaneously")
    print("   Early termination: Stops at first serious violation")
    print("   Cost efficient: Prevents expensive completions")

# Run multi-model streaming demo
if AGENTS_AVAILABLE:
    await demo_multi_model_streaming_guardrails()
else:
    print("Multi-model streaming demo requires Agents SDK")

## Production implementation template

Copy-paste ready code template for your streaming application

In [ ]:
class ProductionStreamingApp:
    """Production template for your streaming application with multi-model guardrails"""
    
    def __init__(self):
        # Initialize multi-model streaming guardrail
        self.streaming_guardrail = MultiModelStreamingGuardrail(check_interval=200)
        
        # Your main production agent
        if AGENTS_AVAILABLE:
            self.sales_agent = Agent(
                name="Production Sales Assistant",
                instructions="""You are a professional sales assistant for a genomics company.
                
                Provide helpful information about:
                • Guardant360, GuardantOMNI, Guardant Reveal products
                • Sales strategies and customer engagement  
                • Technical questions about genomic testing
                
                Always maintain professional tone and avoid sharing:
                • Specific customer personal information
                • Exact pricing details
                • Confidential competitive information
                """,
                model="gpt-4o"
            )
    
    async def safe_streaming_response(self, user_query: str):
        """Main method for streaming with safety guardrails"""
        
        if not AGENTS_AVAILABLE:
            print("Production streaming requires Agents SDK")
            return
        
        print(f"PRODUCTION STREAMING WITH GUARDRAILS")
        print(f"Query: {user_query}")
        print("-" * 60)
        
        # Start streaming
        streaming_result = Runner.run_streamed(self.sales_agent, user_query)
        
        # Monitor with multi-model guardrails
        termination_result = await self.streaming_guardrail.monitor_streaming_response(
            streaming_result,
            user_query
        )
        
        # Return result for your application
        return {
            "terminated": termination_result is not None,
            "termination_reason": termination_result.termination_reason if termination_result else None,
            "safety_status": "BLOCKED" if termination_result else "SAFE",
            "timestamp": datetime.now().isoformat()
        }

# Create production app instance
production_app = ProductionStreamingApp()

print("PRODUCTION STREAMING APP TEMPLATE READY!")
print("   Multi-model streaming guardrails integrated")
print("   Real-time safety monitoring")
print("   Ready for WebSocket/FastAPI integration")
print("   Built-in safety reporting")

# Example usage
print("\nTesting production template:")

test_queries = [
    "What are the features of Guardant360?",  # Should pass
    "Can you share Dr. Smith personal contact information?"  # Should block
]

for query in test_queries:
    print(f"\nTesting: {query}")
    if AGENTS_AVAILABLE:
        result = await production_app.safe_streaming_response(query)
        print(f"Result: {result['safety_status']}")
    else:
        print("Would be tested with full SDK")

## Summary & Integration guide

### Multi-model streaming guardrails I built:
1. **OpenAI Guardrail**: Fast GPT-4o Mini for sensitive data & topic detection
2. **Bedrock Guardrail**: Claude 3.5 Sonnet for professional tone & competitive intel
3. **Streaming Integration**: Real-time monitoring during response generation
4. **Early Termination**: Stop problematic responses mid-generation
5. **Parallel Processing**: Both models check simultaneously for speed

### Your streaming application integration:

```python
# Initialize multi-model guardrail system
streaming_guardrail = MultiModelStreamingGuardrail(check_interval=200)

# Monitor streaming response
async for event in streaming_result.stream_events():
    if len(current_text) >= next_check:
        safety_result = await streaming_guardrail.comprehensive_streaming_check(current_text)
        if safety_result.should_terminate:
            # Terminate streaming and notify frontend
            break
```

### Perfect for streaming applications:

**Real-Time Safety**: Problems caught as they happen during streaming

**Cost Control**: Expensive model calls terminated early when issues detected

**Multi-Model Coverage**: OpenAI + Bedrock catch different violation types

**Production Performance**: Parallel processing keeps checks fast

**Drop-in Integration**: Easy to add to existing streaming applications

### Key benefits:

**For Output Guardrails**: Streaming safety is the ultimate output protection

**For Multi-Model**: Different AI models excel at different safety aspects  

**For Real-Time Apps**: Essential for chat/streaming user experiences

**For Cost Efficiency**: Early termination prevents wasteful completions

---

### Ready for production:
**Multi-Model Safety**: OpenAI + Bedrock comprehensive protection

**Streaming Output Guardrails**: Real-time monitoring as requested

**Production Templates**: Copy-paste ready code

**Early Termination**: Cost-saving safety stops

*Your streaming application now has enterprise-grade, multi-model safety guardrails!*